In [1]:
#!git clone https://github.com/p100mma/turbine_lite

Cloning into 'turbine_lite'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 6 (delta 0), reused 6 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [2]:
import numpy as np
import pandas as pd
import tensorflow.keras as K

# Description

Assumptions:

- one file contains array with one column per one time series (variable, i.e. wind speed) to be used as input to the model (X), one file contains array with one variable to be predicted by the model (Y). Along with time series, one column specifying timestamp is added. Both have same number of rows, number of rows is equal to the number of timesteps under consideration, timesteps are evenly spaced, such that for example, observation at row $t+1$ is 5 min later than observation at row $t$, and row $t$ is 5 min later than $t-1$ etc.

- both Y and X can contain missing values (NaN or null). For Y, an algorithm for fiding avaliable regions for the input to the network should be used that returns indexes of those regions (rows in X and Y). Then, the indexes should be partitioned into dataset portions avaliable for training the model and testing (testing probably should consist of later timestamps than training).

- after necessary preprocessing, sliding window approach is used to handle the data to the neural network model in samples, where each sample consists of pair of (input, target), where:

  - input has defined length in timesteps, call it $pastHistory$,

  - target has defined length in timesteps not necessairly equal to the input length, call it $horizon$,

  - theese parameters should be considered also during the algorithm for finding regions in initial X, Y data avaliable for handling to the network. 

In [34]:
  # Initial X, Y arrays
  #%cd /content/
  X_filename='my_X.csv'
  Y_filename='my_Y.csv'
  X_series=pd.read_csv(X_filename)
  X_series.Timestamp=pd.to_datetime(X_series.Timestamp)
  Y_series=pd.read_csv(Y_filename)
  Y_series.Timestamp=pd.to_datetime(Y_series.Timestamp)
  assert X_series.shape[0]==Y_series.shape[0], "Y and X have different number of rows!"
  Y_series.index=[i for i in range(Y_series.shape[0])]
  X_series.index=Y_series.index


/content


In [26]:
X_series.head()

,Timestamp,dir,v,state,temp
0,2021-01-01 00:00:00.961,5.210624,5.322689,102.0,1.500000
1,2021-01-01 00:05:00.961,6.656118,5.443119,102.0,1.462500
2,2021-01-01 00:10:00.961,7.687749,4.901177,102.0,1.440000
3,2021-01-01 00:15:00.961,8.719379,4.775746,102.0,1.350000
4,2021-01-01 00:20:00.961,1.962600,4.833967,102.0,1.333333


In [5]:
Y_series.head()

,Timestamp,p
0,2021-01-01 00:00:00.961,0.239603
1,2021-01-01 00:05:00.961,0.245508
2,2021-01-01 00:10:00.961,0.217836
3,2021-01-01 00:15:00.961,0.192571
4,2021-01-01 00:20:00.961,0.168903


In [6]:
X_series.tail()

,Timestamp,dir,v,state,temp
16986,2021-02-28 23:30:00.961,3.331437,5.949410,102.0,3.6
16987,2021-02-28 23:35:00.961,3.331437,5.956607,102.0,3.6
16988,2021-02-28 23:40:00.961,1.667115,5.462344,102.0,3.6
16989,2021-02-28 23:45:00.961,1.667115,5.766281,102.0,3.6
16990,2021-02-28 23:50:00.961,-0.871973,6.352681,102.0,3.6


In [7]:
Y_series.tail()

,Timestamp,p
16986,2021-02-28 23:30:00.961,0.345500
16987,2021-02-28 23:35:00.961,0.342889
16988,2021-02-28 23:40:00.961,0.281623
16989,2021-02-28 23:45:00.961,0.309770
16990,2021-02-28 23:50:00.961,0.444568


## Preprocessing step

Consists of:

- adding the Y to the X matrix if past timestamps of Y are to be used in predicting future Y,

- interpolation of X matrix (inputs to the network),

- transformation of X, Y:

  - special transformations, such as converting wind speed and angle to wind vector in cartesian coordinates,
  - standarization/centralization/scaling transformations

- where transformations are applied to the parts specified by training and validation regions prepared before separately.

In [18]:
partitionLY= {"train": [11, 5694], 
        "validation": [11895], 
        "test": [15288]} 
partitionRY= {"train": [5693, 11903],
        "validation": [15296], 
        "test": [16991]} 
partitionLX= {"train": [0, 5683], 
        "validation": [11884], 
        "test": [15277]}
partitionRX={"train": [5684, 11894],
        "validation": [15287], 
        "test": [16982]}

In [35]:

import preprocessor
TransformColsXsargs={
      "X_series": X_series,
      "L": partitionLX["train"] + partitionLX["validation"],
      "R": partitionRX["train"] + partitionRX["validation"],
      "feature_mapping":  {'v':(lambda r,A: r*np.cos(A),['v','dir'],[] ),
                          'dir':(lambda r,A: r*np.sin(A),['v','dir'],[] )
                          } #keys: names of columns

}
X_series= preprocessor.TransformCols(**TransformColsXsargs)
X_series = pd.concat([X_series, Y_series['p'] ], axis=1) #use also past power timestamps

# standardization parameters (mean, sd) are learned only on training set
X_series, X_estimators = preprocessor.AutoIntStanScale(X_series,
                                                       {"p":['interpolate','standardize'],
                                                        "v":['interpolate','standardize'],
                                                        "dir":['interpolate','standardize'],
                                                        "temp":['interpolate','standardize']
                                                        },
                                                        partitionLX, partitionRX, 
                                                        keep_estimators=True)
Y_series, Y_estimators = preprocessor.AutoIntStanScale(Y_series,
                                                       {"p":['standardize']},
                                                        partitionLY, partitionRY, 
                                                        keep_estimators=True)
#keep_estimators==True allows for reverse-transform of validation samples later 


## Model definition

Assumption on the NN model are as follows:

- model has 3 or 4 inital modules, one for each input time-series/variable, which are then linked to output one time-series of interest to predict,

- special SlidingWindow generator is used for handling the samples to the network, which takes into account the training and validation regions specified by indexes mentioned above

- it's relatively easy to simplify the above part if one wants the sliding window to be ran on all of the regions of X and Y matrices (for example where no invalid values are in Y)

In [40]:
#%cd turbine_lite
import handler
# creating the generator object to be used with keras model.fit
X_indexes_tr=[np.arange(partitionLX["train"][i],partitionRX["train"][i]) for i in range(len(partitionLX["train"]))]
Y_indexes_tr=[np.arange(partitionLY["train"][i],partitionRY["train"][i]) for i in range(len(partitionLY["train"]))]
X_indexes_te=[np.arange(partitionLX["validation"][i],partitionRX["validation"][i]) for i in range(len(partitionLX["validation"]))]
Y_indexes_te=[np.arange(partitionLY["validation"][i],partitionRY["validation"][i]) for i in range(len(partitionLY["validation"]))]
batch_size=32
pastHistory, horizon = 11, 9  #will work with example X_indexes, Y_indexes (partitionLX, etc.)!...
#...other pastHistory, horizon parameters will require different set of X and Y indexes
feature_names=["v","dir","temp","p"]
y_names=["p"]
DataHandlerTrain= handler.SlidingWindowManySlices(
                   X_series, Y_series,
                   X_indexes_tr, Y_indexes_tr,
                   batch_size,
                   pastHistory,
                   horizon,
                   feature_names,
                   y_names)
DataHandlerTest= handler.SlidingWindowManySlices(
                   X_series, Y_series,
                   X_indexes_te, Y_indexes_te,
                   batch_size,
                   pastHistory,
                   horizon,
                   feature_names,
                   y_names)

In [48]:
#NN architecture
n_layers=4
n_f=[16 for i in range(n_layers)] #number of filters per layer in each NN module
kernel_size_per_layer=[7,5,3,3] # kernel sizes
do_BN= False # Batch Normalization between conv layers, in my case it didn't improve the results
K.backend.clear_session() #tidy up keras computation graph just in case
#assemble model by joining 4 heads, one per each time series input
ins=[]
heads=[]
for i in range(4): #for each input time-series
  In= K.Input(shape=(pastHistory,1)) #number of inputs in timesteps to the network is always pastHistory
  ins.append(In)
  prev=In
  for layer in range(n_layers):
    new1=K.layers.Conv1D(n_f[layer], kernel_size_per_layer[layer], padding="same")(prev)
    new2=K.layers.ELU(alpha=3.0)(new1)
    new3=K.layers.MaxPool1D(padding="same")(new2)
    if do_BN:
      new4=K.layers.BatchNormalization()(new3)
      prev=new4
    else:
      prev=new3
  heads.append(prev)
Link= K.layers.Concatenate()(heads)
Flatten=K.layers.Flatten()(Link)
FC= K.layers.Dense(100, activation='relu')(Flatten)
Out= K.layers.Dense(horizon, activation='linear')(FC)   #number of outputs= horizon,...
#...also linear activation (identity) in the last layer because of standardization...
#...some target values might be negative 
my_model= K.Model(inputs=ins, outputs=[Out], name="Example.CNN")


## Model training

In [49]:
my_model.compile(loss='mse', 
                metrics=['mae'],
                optimizer=K.optimizers.Adam(learning_rate=0.001))

In [50]:
my_model.summary()

Model: "Example.CNN"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 11, 1)]      0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 11, 1)]      0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 11, 1)]      0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 11, 1)]      0           []                               
                                                                                        

In [51]:
N_EPOCHS=100
HISTORY=my_model.fit(DataHandlerTrain, validation_data=DataHandlerTest, epochs=N_EPOCHS,
                    callbacks=[K.callbacks.EarlyStopping(monitor='val_loss',
                                                         patience=40,
                                                         restore_best_weights=True)]
                  )
pd.DataFrame(HISTORY.history).to_csv('ResultsArray.csv')

Epoch 1/100
370/370 [==============================] - 23s 55ms/step - loss: 0.2894 - mae: 0.3338 - val_loss: 0.1644 - val_mae: 0.2808
Epoch 2/100
148/370 [===========>..................] - ETA: 9s - loss: 0.1890 - mae: 0.2644

KeyboardInterrupt: ignored

In [53]:
targets=np.vstack([batch[1] for batch in DataHandlerTest])

## Model prediction on validation set

In [54]:
predictions= my_model.predict(DataHandlerTest).reshape(targets.shape)
# to be compared with targets array for error values on validation set

106/106 [==============================] - 4s 36ms/step


In [55]:
#power prediction to WATTS or whatever initial units (inverse of standarization)
prediction_watts=np.array(
     [ Y_estimators['standardize']['p'].inverse_transform(sample) for sample in predictions ] 
                         )
targets_watts= np.array(
     [ Y_estimators['standardize']['p'].inverse_transform(sample) for sample in targets ]
                        )

In [57]:
prediction_watts.shape

(3392, 9, 1)

In [58]:
targets_watts.shape

(3392, 9, 1)